# Create lable files 
annottaions are based on 5s (5\*30fps)
features are based on 64 frames (~2.5s)

two things to be done:
- devide the lables to two (2.5s)
- merge the features to one vector (5s) 

Lables are set to likart scale of 9
- want to try 5 scale and 3 scale too

### imports & variables

In [1]:
import sys
import numpy as np
import os
import pandas as pd

lables_path = './labels/'
all_folders_path = './features/'

### help functions

In [2]:
def normalize_labels_regression(dataChange, new_range):
    # regression with lables that are minus don't perfom well, so we scale to a new range
    min_old_range = -4
    max_old_range = 4
    if new_range == 5:
        min_new_range = 0
        max_new_range = 5
    elif new_range == 3:
        min_new_range = 0
        max_new_range = 3
    else:
        min_new_range = 0
        max_new_range = 9
        
    newdataChange = (((dataChange - min_old_range) * (max_new_range - min_new_range))
                     / (max_old_range - min_old_range)) + min_new_range
    return newdataChange
    
def to_five_cat(dataChange):
    dataChange.replace(2,1, inplace=True)
    dataChange.replace(4,2, inplace=True)
    dataChange.replace(3,2, inplace=True)

    dataChange.replace(-2,-1, inplace=True)
    dataChange.replace(-3,-2, inplace=True)
    dataChange.replace(-4,-2, inplace=True)
    return dataChange


def to_three_cat(dataChange):
    dataChange.replace(4,1, inplace=True)
    dataChange.replace(3,1, inplace=True)
    dataChange.replace(2,1, inplace=True)

    dataChange.replace(-2,-1, inplace=True)
    dataChange.replace(-3,-1, inplace=True)
    dataChange.replace(-4,-1, inplace=True)

    return dataChange

def to_two_cat(dataChange):
    dataChange.replace(4,1, inplace=True)
    dataChange.replace(3,1, inplace=True)
    dataChange.replace(2,1, inplace=True)

    dataChange.replace(-1,0, inplace=True)
    dataChange.replace(-2,0, inplace=True)
    dataChange.replace(-3,0, inplace=True)
    dataChange.replace(-4,0, inplace=True)

    return dataChange

def person_new_eng_lable(this_file,all_folders_path, eng_lbl, classType, prblemType, classes=9, divide=2.5):
    # F10_Interaction_1_P27_rgb.npy <-- remove the _rgb.npy or _flow.npy or vggish etc
#     this_bhv = this_file.replace('_'+this_file.split('_')[4],'')
    this_bhv = this_file.replace('_high_level.npy','')
    engag_lbl = eng_lbl[eng_lbl['behaviour'] == this_bhv]
    currData = np.load(os.path.join(all_folders_path,this_file))
    
    #convert lables to frame-level lables
    currLabel = np.repeat(engag_lbl[classType].to_numpy(), repeats=5*30, axis=0)

    #... , then merge into the length of the features
    reshape_span = int(64*2) if divide == 5 else 64
    cutoff = int(currData.shape[0]/2) * reshape_span if divide == 5 else currData.shape[0]*reshape_span
    lostLabel = currLabel[cutoff:]
    currLabel = currLabel[:cutoff]
    
    t=currLabel.shape
    
    if prblemType == 'classification':
        currLabel = np.mean(currLabel.reshape(-1, reshape_span), axis=1).round()
    elif prblemType == 'regression': # <-- not sure about this for now!
        currLabel = np.mean(currLabel.reshape(-1, reshape_span), axis=1)
        
    #print('{} - {} - {} -> {}'.format(this_file,currData.shape[0],t,currLabel.shape))
    
    #folder like: classification_eng_lvl_avg_9_2.5s
    new_label_folder = '_'.join([classType,'eng_lvl',prblemType,str(classes),str(divide)])
    os.makedirs(os.path.join(lables_path,new_label_folder), exist_ok=True)
    
    new_lable_file = this_bhv+'.npy'
    np.save(os.path.join(lables_path,new_label_folder,new_lable_file), currLabel)
           

def create_new_eng_lable(all_folders_path, classType, prblemType, classes, divide):
    eng_lbl = pd.read_csv(os.path.join(lables_path,'Speed_dating_avg_eng.csv'),
                          usecols=['video_file','person', classType])
    
    if classes not in [9,5,3,2]:
        sys.exit('The number of classes you selected is not supported')
        
    if prblemType == 'classification':
        if classes == 5:
            eng_lbl[classType] = to_five_cat(eng_lbl[classType])
        if classes == 3:
            eng_lbl[classType] = to_three_cat(eng_lbl[classType])
        if classes == 2:
            eng_lbl[classType] = to_two_cat(eng_lbl[classType])
    elif prblemType == 'regression':
        eng_lbl[classType] = normalize_labels_regression(eng_lbl[classType], classes)
    
    eng_lbl['person'] = pd.DataFrame(eng_lbl['person'].str.split(' ', n=1).tolist())[0]
    eng_lbl['behaviour'] = eng_lbl['video_file'].str.cat(eng_lbl['person'], sep="_")

    onlyfiles = [f for f in os.listdir(all_folders_path) if
                   os.path.isfile(os.path.join(all_folders_path, f))]
    onlyfiles.sort()

    for this_file in onlyfiles:
        person_new_eng_lable(this_file,all_folders_path, eng_lbl, classType, prblemType, classes, divide)


### create the files

In [3]:
# for prblemType in ['classification', 'regression']:
#     classType = 'round_avg_eng_level' if prblemType == 'classification' else 'avg_eng_level'
#     for classes in [9,5,3]:
#         for divide in [2.5, 5]:
#             print('Working on {}_{}_{}_{}'.format(classType, prblemType, classes, divide))
#             create_new_eng_lable(os.path.join(all_folders_path,'i3d_rgb_features'), 
#                                  classType, prblemType, classes, divide)

### create file for 5s

In [4]:

def person_new_eng_lable_5s(this_file,all_folders_path, eng_lbl, classType, prblemType, classes=9):
    # F10_Interaction_1_P27_rgb.npy <-- remove the _rgb.npy or _flow.npy or vggish etc
    this_bhv = this_file.replace('_high_level.npy','')
    engag_lbl = eng_lbl[eng_lbl['behaviour'] == this_bhv]
    
    currData = np.load(os.path.join(all_folders_path,this_file))
    currLabel = engag_lbl[classType].to_numpy()

    tb=currLabel.shape[0]
    cutoff = currData.shape[0]
    print('{} - {} - {}'.format(this_file,cutoff,tb))
    
    if tb >= cutoff:
        lostLabel = currLabel[cutoff:]
        currLabel = currLabel[:cutoff]
    else:
        lastLabel = currLabel[tb]
        di = cutoff - tb
        ext_lable = np.repeat(lastLabel,di)
        currLabel = np.append(currLabel, ext_lable, axis=0)
    
    print('{} - {} - {} -> {}'.format(this_file,currData.shape[0],tb,currLabel.shape))
    
    #folder like: classification_eng_lvl_avg_9_5s
    new_label_folder = '_'.join([classType,'eng_lvl',prblemType,str(classes),'5s'])
    os.makedirs(os.path.join(lables_path,new_label_folder), exist_ok=True)
    
    new_lable_file = this_bhv+'.npy'
    np.save(os.path.join(lables_path,new_label_folder,new_lable_file), currLabel)
           

def create_new_eng_lable_5s(all_folders_path, classType, prblemType, classes):
    eng_lbl = pd.read_csv(os.path.join(lables_path,'Speed_dating_avg_eng.csv'),
                          usecols=['video_file','person', classType])
    
    if classes not in [9,5,3,2]:
        sys.exit('The number of classes you selected is not supported')
        
    if prblemType == 'classification':
        if classes == 5:
            eng_lbl[classType] = to_five_cat(eng_lbl[classType])
        if classes == 3:
            eng_lbl[classType] = to_three_cat(eng_lbl[classType])
        if classes == 2:
            eng_lbl[classType] = to_two_cat(eng_lbl[classType])
    elif prblemType == 'regression':
        eng_lbl[classType] = normalize_labels_regression(eng_lbl[classType], classes)
    
    eng_lbl['person'] = pd.DataFrame(eng_lbl['person'].str.split(' ', n=1).tolist())[0]
    eng_lbl['behaviour'] = eng_lbl['video_file'].str.cat(eng_lbl['person'], sep="_")

    onlyfiles = [f for f in os.listdir(all_folders_path) if
                   os.path.isfile(os.path.join(all_folders_path, f))]
    onlyfiles.sort()

    for this_file in onlyfiles:
        person_new_eng_lable_5s(this_file,all_folders_path, eng_lbl, classType, prblemType, classes)

# for prblemType in ['classification', 'regression']:
#     classType = 'round_avg_eng_level' if prblemType == 'classification' else 'avg_eng_level'
#     for classes in [9,5,3]:
#         print('Working on {}_{}_{}'.format(classType, prblemType, classes))
#         create_new_eng_lable_5s(os.path.join(all_folders_path,'handcrafted_features_5s'), 
#                              classType, prblemType, classes)

### binary 

In [5]:
for prblemType in ['classification']:
    classType = 'round_avg_eng_level' if prblemType == 'classification' else 'avg_eng_level'
    for classes in [2]:
        for divide in [2.5, 5, '5s']:
            print('Working on {}_{}_{}_{}'.format(classType, prblemType, classes, str(divide)))
            if divide in [2.5, 5]:
                create_new_eng_lable(os.path.join(all_folders_path,'handcrafted_features_'+str(divide)), 
                                     classType, prblemType, classes, divide)
            elif divide == '5s':
                create_new_eng_lable_5s(os.path.join(all_folders_path,'handcrafted_features_5s'), 
                             classType, prblemType, classes)


Working on round_avg_eng_level_classification_2_2.5
Working on round_avg_eng_level_classification_2_5
Working on round_avg_eng_level_classification_2_5s
F10_Interaction_1_P27_high_level.npy - 213 - 213
F10_Interaction_1_P27_high_level.npy - 213 - 213 -> (213,)
F10_Interaction_1_P28_high_level.npy - 211 - 213
F10_Interaction_1_P28_high_level.npy - 211 - 213 -> (211,)
F11_Interaction_1_P29_high_level.npy - 212 - 213
F11_Interaction_1_P29_high_level.npy - 212 - 213 -> (212,)
F11_Interaction_1_P30_high_level.npy - 212 - 213
F11_Interaction_1_P30_high_level.npy - 212 - 213 -> (212,)
F11_Interaction_2_P29_high_level.npy - 161 - 213
F11_Interaction_2_P29_high_level.npy - 161 - 213 -> (161,)
F11_Interaction_2_P30_high_level.npy - 184 - 213
F11_Interaction_2_P30_high_level.npy - 184 - 213 -> (184,)
F13_Interaction_1_P32_high_level.npy - 213 - 213
F13_Interaction_1_P32_high_level.npy - 213 - 213 -> (213,)
F13_Interaction_1_P33_high_level.npy - 213 - 213
F13_Interaction_1_P33_high_level.npy - 213